# Imports


In [1]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import joblib
import random
import sys
import os
import re

from rapidfuzz import fuzz
from rapidfuzz.distance import Levenshtein
import unicodedata

from ann_levenshtein import LevenshteinIndex

# Data


### Random Strings


In [2]:
strings = [
    "AAA", "AAB", "AAC", "ABA", 
    "ABB", "ABC", "ACA", "ACB",
    "ACC", "BAA", "BAB", "BAC", 
    "BBA", "BBB", "BBC", "BCA",
    "BCB", "BCC", "CAA", "CAB", 
    "CAC", "CBA", "CBB", "CBC",
    "CCA", "CCB", "CCC", "ABCA", 
    "BABC", "CABA", "ACBC","CBAC"
]

### function


In [ ]:
def Levenshtein_dist_vector(s1, s2):
        insertions, deletions, substitutions = 0, 0, 0
        for tag, _, _ in Levenshtein.editops(s1, s2):
            if tag == 'insert':
                insertions += 1
            elif tag == 'delete':
                deletions += 1
            elif tag == 'replace':
                substitutions += 1
        return (insertions, deletions, substitutions)

# Build tree


In [3]:
indices = np.arange(len(strings))

In [4]:
# Initialize
estimated_nodes = 8 * len(strings) + 8

tree_s1 = np.full(shape=estimated_nodes, fill_value=-1, dtype=np.int32)
tree_s2 = np.full(shape=estimated_nodes, fill_value=-1, dtype=np.int32)

tree_ppp = np.full(shape=estimated_nodes, fill_value=-1, dtype=np.int32)
tree_ppm = np.full(shape=estimated_nodes, fill_value=-1, dtype=np.int32)
tree_pmp = np.full(shape=estimated_nodes, fill_value=-1, dtype=np.int32)
tree_pmm = np.full(shape=estimated_nodes, fill_value=-1, dtype=np.int32)
tree_mpp = np.full(shape=estimated_nodes, fill_value=-1, dtype=np.int32)
tree_mpm = np.full(shape=estimated_nodes, fill_value=-1, dtype=np.int32)
tree_mmp = np.full(shape=estimated_nodes, fill_value=-1, dtype=np.int32)
tree_mmm = np.full(shape=estimated_nodes, fill_value=-1, dtype=np.int32)
leaf_value = np.full(shape=estimated_nodes, fill_value=-1, dtype=np.int32)

In [5]:
tree_s1

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1

In [53]:
# queue
queue = [(0, np.arange(len(strings)))]  # (idx, indices)
node_counter = 1


In [54]:
queue

[(0,
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]))]

### node 0


In [55]:
current_node_id, indices = queue.pop(0)

In [56]:
s1_idx, s2_idx = np.random.choice(indices, 2, replace=False)
s1, s2 = strings[s1_idx], strings[s2_idx]

In [57]:
s1, s2

('ABC', 'ABA')

In [58]:
mask_ppp = np.full(len(indices), False, dtype=bool)
mask_ppm = np.full(len(indices), False, dtype=bool)
mask_pmp = np.full(len(indices), False, dtype=bool)
mask_pmm = np.full(len(indices), False, dtype=bool)
mask_mpp = np.full(len(indices), False, dtype=bool)
mask_mpm = np.full(len(indices), False, dtype=bool)
mask_mmp = np.full(len(indices), False, dtype=bool)
mask_mmm = np.full(len(indices), False, dtype=bool)

In [60]:
for j, idx in enumerate(indices):
    each_string = strings[idx]
    (a1, a2, a3) = Levenshtein_dist_vector(each_string, s1)
    (b1, b2, b3) = Levenshtein_dist_vector(each_string, s2)
    # split into 8 parts
    if a1>=b1 and a2>=b2 and a3>=b3:  # +++
        mask_ppp[j] = True  # go tree_ppp
    elif a1>=b1 and a2>=b2 and a3<b3:  # ++-
        mask_ppm[j] = True  # go tree_ppm
    elif a1>=b1 and a2<b2 and a3>=b3:  # +-+
        mask_pmp[j] = True  # go tree_pmp
    elif a1>=b1 and a2<b2 and a3<b3:  # +--
        mask_pmm[j] = True  # go tree_pmm
    elif a1<b1 and a2>=b2 and a3>=b3:  # -++
        mask_mpp[j] = True  # go tree_mpp
    elif a1<b1 and a2>=b2 and a3<b3:  # -+-
        mask_mpm[j] = True  # go tree_mpm
    elif a1<b1 and a2<b2 and a3>=b3:  # --+
        mask_mmp[j] = True  # go tree_mmp
    else:  # ---
        mask_mmm[j] = True  # go tree_mmm

In [61]:
mask_ppp

array([ True, False, False,  True,  True, False, False, False, False,
        True,  True,  True,  True,  True, False,  True, False, False,
        True,  True, False,  True,  True, False, False, False, False,
        True, False,  True, False,  True])

In [62]:
indices_ppp = indices[mask_ppp]
indices_ppm = indices[mask_ppm]
indices_pmp = indices[mask_pmp]
indices_pmm = indices[mask_pmm]
indices_mpp = indices[mask_mpp]
indices_mpm = indices[mask_mpm]
indices_mmp = indices[mask_mmp]
indices_mmm = indices[mask_mmm]

In [63]:
indices_ppp, indices_ppm, indices_pmp, indices_pmm, indices_mpp, indices_mpm, indices_mmp, indices_mmm

(array([ 0,  3,  4,  9, 10, 11, 12, 13, 15, 18, 19, 21, 22, 27, 29, 31]),
 array([ 5,  6,  7,  8, 14, 16, 17, 23, 24, 25, 26, 28, 30]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([ 1,  2, 20]),
 array([], dtype=int64))

In [64]:
tree_s1[current_node_id] = s1_idx
tree_s2[current_node_id] = s2_idx

In [65]:
id_ppp = node_counter
id_ppm = node_counter + 1
id_pmp = node_counter + 2
id_pmm = node_counter + 3
id_mpp = node_counter + 4
id_mpm = node_counter + 5
id_mmp = node_counter + 6
id_mmm = node_counter + 7
node_counter += 8

In [66]:
tree_ppp[current_node_id] = id_ppp
tree_ppm[current_node_id] = id_ppm
tree_pmp[current_node_id] = id_pmp
tree_pmm[current_node_id] = id_pmm
tree_mpp[current_node_id] = id_mpp
tree_mpm[current_node_id] = id_mpm
tree_mmp[current_node_id] = id_mmp
tree_mmm[current_node_id] = id_mmm

In [67]:
queue.append((id_ppp, indices_ppp))
queue.append((id_ppm, indices_ppm))
queue.append((id_pmp, indices_pmp))
queue.append((id_pmm, indices_pmm))
queue.append((id_mpp, indices_mpp))
queue.append((id_mpm, indices_mpm))
queue.append((id_mmp, indices_mmp))
queue.append((id_mmm, indices_mmm))

In [68]:
queue

[(1, array([ 0,  3,  4,  9, 10, 11, 12, 13, 15, 18, 19, 21, 22, 27, 29, 31])),
 (2, array([ 5,  6,  7,  8, 14, 16, 17, 23, 24, 25, 26, 28, 30])),
 (3, array([], dtype=int64)),
 (4, array([], dtype=int64)),
 (5, array([], dtype=int64)),
 (6, array([], dtype=int64)),
 (7, array([ 1,  2, 20])),
 (8, array([], dtype=int64))]

### node 1


In [69]:
current_node_id, indices = queue.pop(0)

In [70]:
current_node_id, indices

(1, array([ 0,  3,  4,  9, 10, 11, 12, 13, 15, 18, 19, 21, 22, 27, 29, 31]))

In [71]:
s1_idx, s2_idx = np.random.choice(indices, 2, replace=False)
s1, s2 = strings[s1_idx], strings[s2_idx]

In [72]:
s1, s2

('CAA', 'CBA')

In [73]:
mask_ppp = np.full(len(indices), False, dtype=bool)
mask_ppm = np.full(len(indices), False, dtype=bool)
mask_pmp = np.full(len(indices), False, dtype=bool)
mask_pmm = np.full(len(indices), False, dtype=bool)
mask_mpp = np.full(len(indices), False, dtype=bool)
mask_mpm = np.full(len(indices), False, dtype=bool)
mask_mmp = np.full(len(indices), False, dtype=bool)
mask_mmm = np.full(len(indices), False, dtype=bool)

In [75]:
for j, idx in enumerate(indices):
    each_string = strings[idx]
    (a1, a2, a3) = Levenshtein_dist_vector(each_string, s1)
    (b1, b2, b3) = Levenshtein_dist_vector(each_string, s2)
    # split into 8 parts
    if a1>=b1 and a2>=b2 and a3>=b3:  # +++
        mask_ppp[j] = True  # go tree_ppp
    elif a1>=b1 and a2>=b2 and a3<b3:  # ++-
        mask_ppm[j] = True  # go tree_ppm
    elif a1>=b1 and a2<b2 and a3>=b3:  # +-+
        mask_pmp[j] = True  # go tree_pmp
    elif a1>=b1 and a2<b2 and a3<b3:  # +--
        mask_pmm[j] = True  # go tree_pmm
    elif a1<b1 and a2>=b2 and a3>=b3:  # -++
        mask_mpp[j] = True  # go tree_mpp
    elif a1<b1 and a2>=b2 and a3<b3:  # -+-
        mask_mpm[j] = True  # go tree_mpm
    elif a1<b1 and a2<b2 and a3>=b3:  # --+
        mask_mmp[j] = True  # go tree_mmp
    else:  # ---
        mask_mmm[j] = True  # go tree_mmm

In [76]:
mask_ppp

array([False, False, False, False, False, False,  True,  True, False,
       False, False,  True,  True, False,  True,  True])

In [77]:
indices_ppp = indices[mask_ppp]
indices_ppm = indices[mask_ppm]
indices_pmp = indices[mask_pmp]
indices_pmm = indices[mask_pmm]
indices_mpp = indices[mask_mpp]
indices_mpm = indices[mask_mpm]
indices_mmp = indices[mask_mmp]
indices_mmm = indices[mask_mmm]

In [78]:
indices_ppp, indices_ppm, indices_pmp, indices_pmm, indices_mpp, indices_mpm, indices_mmp, indices_mmm

(array([12, 13, 21, 22, 29, 31]),
 array([ 0,  3,  4, 18, 27]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([ 9, 10, 11, 15, 19]),
 array([], dtype=int64))

In [79]:
tree_s1[current_node_id] = s1_idx
tree_s2[current_node_id] = s2_idx

In [80]:
id_ppp = node_counter
id_ppm = node_counter + 1
id_pmp = node_counter + 2
id_pmm = node_counter + 3
id_mpp = node_counter + 4
id_mpm = node_counter + 5
id_mmp = node_counter + 6
id_mmm = node_counter + 7
node_counter += 8

In [81]:
tree_ppp[current_node_id] = id_ppp
tree_ppm[current_node_id] = id_ppm
tree_pmp[current_node_id] = id_pmp
tree_pmm[current_node_id] = id_pmm
tree_mpp[current_node_id] = id_mpp
tree_mpm[current_node_id] = id_mpm
tree_mmp[current_node_id] = id_mmp
tree_mmm[current_node_id] = id_mmm

In [82]:
queue.append((id_ppp, indices_ppp))
queue.append((id_ppm, indices_ppm))
queue.append((id_pmp, indices_pmp))
queue.append((id_pmm, indices_pmm))
queue.append((id_mpp, indices_mpp))
queue.append((id_mpm, indices_mpm))
queue.append((id_mmp, indices_mmp))
queue.append((id_mmm, indices_mmm))

In [83]:
queue

[(2, array([ 5,  6,  7,  8, 14, 16, 17, 23, 24, 25, 26, 28, 30])),
 (3, array([], dtype=int64)),
 (4, array([], dtype=int64)),
 (5, array([], dtype=int64)),
 (6, array([], dtype=int64)),
 (7, array([ 1,  2, 20])),
 (8, array([], dtype=int64)),
 (9, array([12, 13, 21, 22, 29, 31])),
 (10, array([ 0,  3,  4, 18, 27])),
 (11, array([], dtype=int64)),
 (12, array([], dtype=int64)),
 (13, array([], dtype=int64)),
 (14, array([], dtype=int64)),
 (15, array([ 9, 10, 11, 15, 19])),
 (16, array([], dtype=int64))]

### node 2


In [85]:
current_node_id, indices = queue.pop(0)

In [86]:
current_node_id, indices

(2, array([ 5,  6,  7,  8, 14, 16, 17, 23, 24, 25, 26, 28, 30]))

In [ ]:
# end as empty leaf
if len(indices) == 0:
    pass

In [ ]:
# end as a leaf
if len(indices) == 1:
    leaf_value[current_node_id] = indices[0]

In [87]:
s1_idx, s2_idx = np.random.choice(indices, 2, replace=False)
s1, s2 = strings[s1_idx], strings[s2_idx]

In [88]:
s1, s2

('ACBC', 'ABC')

In [89]:
mask_ppp = np.full(len(indices), False, dtype=bool)
mask_ppm = np.full(len(indices), False, dtype=bool)
mask_pmp = np.full(len(indices), False, dtype=bool)
mask_pmm = np.full(len(indices), False, dtype=bool)
mask_mpp = np.full(len(indices), False, dtype=bool)
mask_mpm = np.full(len(indices), False, dtype=bool)
mask_mmp = np.full(len(indices), False, dtype=bool)
mask_mmm = np.full(len(indices), False, dtype=bool)

In [90]:
for j, idx in enumerate(indices):
    each_string = strings[idx]
    (a1, a2, a3) = Levenshtein_dist_vector(each_string, s1)
    (b1, b2, b3) = Levenshtein_dist_vector(each_string, s2)
    # split into 8 parts
    if a1>=b1 and a2>=b2 and a3>=b3:  # +++
        mask_ppp[j] = True  # go tree_ppp
    elif a1>=b1 and a2>=b2 and a3<b3:  # ++-
        mask_ppm[j] = True  # go tree_ppm
    elif a1>=b1 and a2<b2 and a3>=b3:  # +-+
        mask_pmp[j] = True  # go tree_pmp
    elif a1>=b1 and a2<b2 and a3<b3:  # +--
        mask_pmm[j] = True  # go tree_pmm
    elif a1<b1 and a2>=b2 and a3>=b3:  # -++
        mask_mpp[j] = True  # go tree_mpp
    elif a1<b1 and a2>=b2 and a3<b3:  # -+-
        mask_mpm[j] = True  # go tree_mpm
    elif a1<b1 and a2<b2 and a3>=b3:  # --+
        mask_mmp[j] = True  # go tree_mmp
    else:  # ---
        mask_mmm[j] = True  # go tree_mmm

In [91]:
mask_ppp

array([ True, False, False, False,  True, False, False, False, False,
       False, False, False, False])

In [92]:
indices_ppp = indices[mask_ppp]
indices_ppm = indices[mask_ppm]
indices_pmp = indices[mask_pmp]
indices_pmm = indices[mask_pmm]
indices_mpp = indices[mask_mpp]
indices_mpm = indices[mask_mpm]
indices_mmp = indices[mask_mmp]
indices_mmm = indices[mask_mmm]

In [93]:
indices_ppp, indices_ppm, indices_pmp, indices_pmm, indices_mpp, indices_mpm, indices_mmp, indices_mmm

(array([ 5, 14]),
 array([ 8, 23, 24, 26]),
 array([ 6,  7, 16, 17, 25, 28, 30]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64))

In [94]:
tree_s1[current_node_id] = s1_idx
tree_s2[current_node_id] = s2_idx

In [95]:
id_ppp = node_counter
id_ppm = node_counter + 1
id_pmp = node_counter + 2
id_pmm = node_counter + 3
id_mpp = node_counter + 4
id_mpm = node_counter + 5
id_mmp = node_counter + 6
id_mmm = node_counter + 7
node_counter += 8

In [96]:
tree_ppp[current_node_id] = id_ppp
tree_ppm[current_node_id] = id_ppm
tree_pmp[current_node_id] = id_pmp
tree_pmm[current_node_id] = id_pmm
tree_mpp[current_node_id] = id_mpp
tree_mpm[current_node_id] = id_mpm
tree_mmp[current_node_id] = id_mmp
tree_mmm[current_node_id] = id_mmm

In [97]:
queue.append((id_ppp, indices_ppp))
queue.append((id_ppm, indices_ppm))
queue.append((id_pmp, indices_pmp))
queue.append((id_pmm, indices_pmm))
queue.append((id_mpp, indices_mpp))
queue.append((id_mpm, indices_mpm))
queue.append((id_mmp, indices_mmp))
queue.append((id_mmm, indices_mmm))

In [98]:
queue

[(3, array([], dtype=int64)),
 (4, array([], dtype=int64)),
 (5, array([], dtype=int64)),
 (6, array([], dtype=int64)),
 (7, array([ 1,  2, 20])),
 (8, array([], dtype=int64)),
 (9, array([12, 13, 21, 22, 29, 31])),
 (10, array([ 0,  3,  4, 18, 27])),
 (11, array([], dtype=int64)),
 (12, array([], dtype=int64)),
 (13, array([], dtype=int64)),
 (14, array([], dtype=int64)),
 (15, array([ 9, 10, 11, 15, 19])),
 (16, array([], dtype=int64)),
 (17, array([ 5, 14])),
 (18, array([ 8, 23, 24, 26])),
 (19, array([ 6,  7, 16, 17, 25, 28, 30])),
 (20, array([], dtype=int64)),
 (21, array([], dtype=int64)),
 (22, array([], dtype=int64)),
 (23, array([], dtype=int64)),
 (24, array([], dtype=int64))]

### node 3


In [99]:
current_node_id, indices = queue.pop(0)

In [100]:
current_node_id, indices

(3, array([], dtype=int64))

In [102]:
# end as empty leaf
if len(indices) == 0:
    pass

### node 4


In [103]:
current_node_id, indices = queue.pop(0)

In [104]:
current_node_id, indices

(4, array([], dtype=int64))

In [106]:
# end as empty leaf
if len(indices) == 0:
    pass

### node 5


In [ ]:
current_node_id, indices = queue.pop(0)

In [ ]:
current_node_id, indices

(2, array([ 5,  6,  7,  8, 14, 16, 17, 23, 24, 25, 26, 28, 30]))

In [ ]:
# end as empty leaf
if len(indices) == 0:
    pass

In [ ]:
# end as a leaf
if len(indices) == 1:
    leaf_value[current_node_id] = indices[0]

In [ ]:
s1_idx, s2_idx = np.random.choice(indices, 2, replace=False)
s1, s2 = strings[s1_idx], strings[s2_idx]

In [ ]:
s1, s2

('ACBC', 'ABC')

In [ ]:
mask_ppp = np.full(len(indices), False, dtype=bool)
mask_ppm = np.full(len(indices), False, dtype=bool)
mask_pmp = np.full(len(indices), False, dtype=bool)
mask_pmm = np.full(len(indices), False, dtype=bool)
mask_mpp = np.full(len(indices), False, dtype=bool)
mask_mpm = np.full(len(indices), False, dtype=bool)
mask_mmp = np.full(len(indices), False, dtype=bool)
mask_mmm = np.full(len(indices), False, dtype=bool)

In [ ]:
for j, idx in enumerate(indices):
    each_string = strings[idx]
    (a1, a2, a3) = Levenshtein_dist_vector(each_string, s1)
    (b1, b2, b3) = Levenshtein_dist_vector(each_string, s2)
    # split into 8 parts
    if a1>=b1 and a2>=b2 and a3>=b3:  # +++
        mask_ppp[j] = True  # go tree_ppp
    elif a1>=b1 and a2>=b2 and a3<b3:  # ++-
        mask_ppm[j] = True  # go tree_ppm
    elif a1>=b1 and a2<b2 and a3>=b3:  # +-+
        mask_pmp[j] = True  # go tree_pmp
    elif a1>=b1 and a2<b2 and a3<b3:  # +--
        mask_pmm[j] = True  # go tree_pmm
    elif a1<b1 and a2>=b2 and a3>=b3:  # -++
        mask_mpp[j] = True  # go tree_mpp
    elif a1<b1 and a2>=b2 and a3<b3:  # -+-
        mask_mpm[j] = True  # go tree_mpm
    elif a1<b1 and a2<b2 and a3>=b3:  # --+
        mask_mmp[j] = True  # go tree_mmp
    else:  # ---
        mask_mmm[j] = True  # go tree_mmm

In [ ]:
mask_ppp

array([ True, False, False, False,  True, False, False, False, False,
       False, False, False, False])

In [ ]:
indices_ppp = indices[mask_ppp]
indices_ppm = indices[mask_ppm]
indices_pmp = indices[mask_pmp]
indices_pmm = indices[mask_pmm]
indices_mpp = indices[mask_mpp]
indices_mpm = indices[mask_mpm]
indices_mmp = indices[mask_mmp]
indices_mmm = indices[mask_mmm]

In [ ]:
indices_ppp, indices_ppm, indices_pmp, indices_pmm, indices_mpp, indices_mpm, indices_mmp, indices_mmm

(array([ 5, 14]),
 array([ 8, 23, 24, 26]),
 array([ 6,  7, 16, 17, 25, 28, 30]),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64))

In [ ]:
tree_s1[current_node_id] = s1_idx
tree_s2[current_node_id] = s2_idx

In [ ]:
id_ppp = node_counter
id_ppm = node_counter + 1
id_pmp = node_counter + 2
id_pmm = node_counter + 3
id_mpp = node_counter + 4
id_mpm = node_counter + 5
id_mmp = node_counter + 6
id_mmm = node_counter + 7
node_counter += 8

In [ ]:
tree_ppp[current_node_id] = id_ppp
tree_ppm[current_node_id] = id_ppm
tree_pmp[current_node_id] = id_pmp
tree_pmm[current_node_id] = id_pmm
tree_mpp[current_node_id] = id_mpp
tree_mpm[current_node_id] = id_mpm
tree_mmp[current_node_id] = id_mmp
tree_mmm[current_node_id] = id_mmm

In [ ]:
queue.append((id_ppp, indices_ppp))
queue.append((id_ppm, indices_ppm))
queue.append((id_pmp, indices_pmp))
queue.append((id_pmm, indices_pmm))
queue.append((id_mpp, indices_mpp))
queue.append((id_mpm, indices_mpm))
queue.append((id_mmp, indices_mmp))
queue.append((id_mmm, indices_mmm))

In [ ]:
queue

[(3, array([], dtype=int64)),
 (4, array([], dtype=int64)),
 (5, array([], dtype=int64)),
 (6, array([], dtype=int64)),
 (7, array([ 1,  2, 20])),
 (8, array([], dtype=int64)),
 (9, array([12, 13, 21, 22, 29, 31])),
 (10, array([ 0,  3,  4, 18, 27])),
 (11, array([], dtype=int64)),
 (12, array([], dtype=int64)),
 (13, array([], dtype=int64)),
 (14, array([], dtype=int64)),
 (15, array([ 9, 10, 11, 15, 19])),
 (16, array([], dtype=int64)),
 (17, array([ 5, 14])),
 (18, array([ 8, 23, 24, 26])),
 (19, array([ 6,  7, 16, 17, 25, 28, 30])),
 (20, array([], dtype=int64)),
 (21, array([], dtype=int64)),
 (22, array([], dtype=int64)),
 (23, array([], dtype=int64)),
 (24, array([], dtype=int64))]